# Titanic Disaster: Survivability Parameters

* [Problem Statement](#problem-statement)

## Problem Statement

* Using data analysis methods, predict which metric or combination of metrics best predict passenger survivability.
* A combination of data visualizations and statistics will be used to determine the most significant predictors of survivability.

### Dataset Description ([data/titanic.csv](data/titanic.csv))

| Variable     | Definition                          | Key                                      |
|--------------|-------------------------------------|------------------------------------------|
| PassengerId  | Passenger ID                        | NA                                       |
| Survival     | Survival                            | 0 = No, 1 = Yes                          |
| Pclass       | Ticket class                        | 1 = 1st, 2 = 2nd, 3 = 3rd                |
| Name         | Passenger name                      | NA                                       |
| Sex          | Sex                                 | NA                                       |
| Age          | Age in years                        | NA                                       |
| SibSp        | # of siblings / spouses aboard the Titanic | NA                                |
| Parch        | # of parents / children aboard the Titanic | NA                                |
| Ticket       | Ticket number                       | NA                                       |
| Fare         | Passenger fare                      | NA                                       |
| Cabin        | Cabin number                        | NA                                       |
| Embarked     | Port of Embarkation                 | C = Cherbourg, Q = Queenstown, S = Southampton |
